In [ ]:


from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import etree
import time 
import requests


#path = 'C:/dd/chromedriver.exe'  # chromedriver.exe 驱动的路径

#sys.stdout = io.TextIOWrapper(sys.stdout.buffer,encoding='gb18030')
#打开浏览器
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
 
# 创建浏览器对象
driver = webdriver.Chrome(chrome_options=chrome_options)

keywords = "unsafe+keyword" 
#略过的文件数量
loss_num = 0
#爬取的文件数量
find_num = 0
for page in range(1,50):
    url='https://github.com/search?p='+str(page)+'&q='+keywords+'&type=Commits'
    #print(url)
    r = requests.get(url)
    select = etree.HTML(r.text)
    result_url_list = select.xpath('//*[@id="commit_search_results"]/div/div/div[@class = "f4 text-normal"]/a[@class = "message markdown-title js-navigation-open"][1]/@href')
    #print(result_url_list)
    for j in range(len(result_url_list)):
        child_url = "https://github.com" + result_url_list[j]
        
        print(child_url)
        driver.get(child_url)

        html = driver.page_source
        selector = etree.HTML(html)

        number = selector.xpath('count(/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"])')


        for next_block in range(1,int(number) + 1): 
            #点击那个按钮，使代码文件全部展开
            Panduan = selector.xpath('/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[1]/div[1]/div/button[1]/@class')
            if "".join(Panduan) == "btn-link color-text-tertiary no-underline js-expand-full directional-expander tooltipped tooltipped-se":
                driver.find_element_by_xpath('/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[1]/div[1]/div/button[1]').click()
                #点击完等浏览器反应
                time.sleep(5)

                html = driver.page_source
                se = etree.HTML(html)



                #得到tr数量
                count = se.xpath('count(/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[2]/div/table/tbody/tr)')

                #print(count)
                #找出所有代码
                text_all = ""
                temp = ""
                for i in range(1,int(count)+1):
                    flag_a = se.xpath('/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[2]/div/table/tbody/tr[' + str(i) + ']/@class')
                    #将list转化为str
                    flag = "".join(flag_a)
                    #print(flag)
                    if flag == "js-expandable-line":
                        continue
                    elif flag == "blob-expanded":
                        text = se.xpath('/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[2]/div/table/tbody/tr[' + str(i) + ']/td[3]//text()')        
                        text_all = text_all + ' '.join(text) + '\n'
                        #print("--------------------------------------------------",text)
                    else:
                        text = se.xpath('/html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[2]/div/table/tbody/tr[' + str(i) + ']/td[3]/span/@data-code-marker | /html/body/div[4]/div/main/div[2]/div/div[@id="files"]/div/div[@data-file-type = ".rs"][' + str(next_block) + ']/div[2]/div/table/tbody/tr[' + str(i) + ']/td[3]//text()')
                        #print("---------------",text)
                        text_all = text_all + ' '.join(text)
                    if temp != "blob-expanded" and flag == "blob-expanded":
                        text_all = text_all + "\n"
                    temp = flag


                #print("text_all = \n",text_all)

                f1 = open("F:/Python/result_1/" + str(next_block) + result_url_list[j].replace("/","_") + "_0.rs", "w", encoding='utf-8') #修改前
                f2 = open("F:/Python/result_1/" + str(next_block) + result_url_list[j].replace("/","_") + "_1.rs", "w", encoding='utf-8') #修改后

                # 1.转换成一行一行的
                # 2.分为修改前的与修改后的文件

                temp_str = ""
                text_list = []
                for i in range(len(text_all)):
                    if text_all[i] != '\n':
                        temp_str = temp_str + text_all[i]
                    else:
                        if temp_str.isspace() == False:
                            if temp_str != "":
                                if len(temp_str) >= 6 and temp_str[5] == "-":
                                    f1.write(temp_str[6:] + "\n")
                                elif len(temp_str) >= 6 and temp_str[5] == '+':
                                    f2.write(temp_str[6:] + "\n")
                                else:
                                    #print(temp_str)
                                    f1.write(temp_str + "\n")
                                    f2.write(temp_str + "\n")
                                #text_list.append(temp_str)
                        temp_str = ""
                f1.close()
                f2.close()
                find_num = find_num + 1
            else:
                loss_num = loss_num + 1
            #print("OK...")
print("OK...")
    
print("loss_num = ",loss_num)
print("find_num = ",find_num)

